In [1]:
import pandas as pd
pd.options.display.max_colwidth = 10
traindf=[]
traindf = pd.read_csv('labeledTrainData.tsv', delimiter="\t")
traindf = traindf.drop(['id'], axis=1)
traindf['type'] = 'train'
addtldf=[]
addtldf = pd.read_csv('unlabeledTrainData.tsv',error_bad_lines=False,delimiter="\t")
addtldf = addtldf.drop(['id'], axis=1)
addtldf['type'] = 'addtl'
revcorpus = pd.concat([traindf,addtldf],ignore_index=True)
print("Number of reviews: ",len(revcorpus))

b'Skipping line 43043: expected 2 fields, saw 3\n'


Number of reviews:  74998


C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [2]:
import re
import string
# import the inflect library 
import inflect 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.util import ngrams,bigrams
from nltk.tokenize import sent_tokenize

stemmer = PorterStemmer() 
p = inflect.engine() 
stop_words = set(stopwords.words("english")) 
# extract pure text from html
def remove_tags(text):
    text = re.sub(r'<[^<>]+>', " ", text)
    return text
# remove stopwords function 
def remove_stopwords(text): 
    word_tokens = word_tokenize(text) 
    filtered_text = [word for word in word_tokens if word not in stop_words] 
    filtered_text = " ".join(filtered_text)
    return filtered_text 

def text_lowercase(text): 
    return text.lower() 
  
# convert number into words 
def convert_number(text): 
    # split string into list of words 
    temp_str = text.split() 
    # initialise empty list 
    new_string = [] 
  
    for word in temp_str: 
        # if word is a digit, convert the digit 
        # to numbers and append into the new_string list 
        if word.isdigit(): 
            temp = p.number_to_words(word) 
            new_string.append(temp) 
  
        # append the word as it is 
        else: 
            new_string.append(word) 
  
    # join the words of new_string to form a string 
    temp_str = ' '.join(new_string) 
    return temp_str

# remove punctuation 
def remove_punctuation(text): 
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator)

    
# remove whitespace from text 
def remove_whitespace(text): 
    return  " ".join(text.split()) 

lemmatizer = WordNetLemmatizer() 
# lemmatize string 
def lemmatize_word(text): 
    word_tokens = word_tokenize(text) 
    # provide context i.e. part-of-speech 
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens] 
    #lemmas = " ".join(lemmas)
    return lemmas 
# stem words in the list of tokenised words 
def stem_words(text): 
    word_tokens = word_tokenize(text) 
    stems = [stemmer.stem(word) for word in word_tokens] 
    stems = " ".join(stems)
    return stems 

def generate_ngram(text,n):
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    tokens = word_tokenize(text) 
    output = list(ngrams(tokens, n))
    #output = bigrams(tokens)
    return output

def preprocess_text(text):
    text = remove_tags(text)
    text = convert_number(text)
    text = text_lowercase(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = remove_whitespace(text)
    #text = stem_words(text)
    #text = ngram_vector(text)
    #print(text)
    text = lemmatize_word(text)
    return text
#revcorpus = revcorpus[1:5]
revcorpus['processed_review'] = revcorpus.review.apply(lambda x: preprocess_text(x))  
#revcorpus['bigrams'] = revcorpus.processed_review.apply(lambda x:generate_ngram(x,2))
print("Number of reviews: ",len(revcorpus))
print(revcorpus.head(3))
#traindf['processed_review'] = traindf.review.apply(lambda x: preprocess_text(x))  
#print("Number of reviews: ",len(traindf))
#traindf.head(3)

Number of reviews:  74998
      review  sentiment   type processed_review
0  With a...        1.0  train  [stuff...      
1  \The C...        1.0  train  [class...      
2  The fi...        0.0  train  [film,...      


In [3]:
from gensim.models import Phrases
reviews = revcorpus['processed_review']
bigrams = Phrases(sentences=reviews)
trigrams = Phrases(sentences=bigrams[reviews])

In [4]:
print(bigrams[reviews][2])
print(trigrams[bigrams[reviews]][2])

['film', 'start', 'manager', 'nicholas', 'bell', 'give', 'welcome', 'investors', 'robert_carradine', 'primal', 'park', 'secret', 'project', 'mutate', 'primal', 'animal', 'use', 'fossilize', 'dna', 'like', '¨jurassik', 'park¨', 'scientists', 'resurrect', 'one', 'natures', 'fearsome', 'predators', 'sabretooth', 'tiger', 'smilodon', 'scientific', 'ambition', 'turn', 'deadly', 'however', 'high_voltage', 'fence', 'open', 'creature', 'escape', 'begin', 'savagely', 'stalk_prey', 'human', 'visitors', 'tourists', 'scientificmeanwhile', 'youngsters', 'enter', 'restrict_area', 'security', 'center', 'attack', 'pack', 'large', 'prehistorical', 'animals', 'deadlier', 'bigger', 'addition', 'security', 'agent', 'stacy', 'haiduk', 'mate', 'brian', 'wimmer', 'fight', 'hardly', 'carnivorous', 'smilodons', 'sabretooths', 'course', 'real', 'star', 'star', 'astound', 'terrifyingly', 'though', 'convince', 'giant', 'animals', 'savagely', 'stalk_prey', 'group', 'run_afoul', 'fight', 'one', 'natures', 'fearsome

In [5]:
#Word2Vec MODEL
from gensim.models import Word2Vec
embedding_vector_size = 300
#tokens = word_tokenize(bigrams[revcorpus['processed_review']])
wordvec_model = Word2Vec(
    sentences = trigrams[bigrams[reviews]],
    size = embedding_vector_size,
    min_count=3, window=5, workers=4,sg=1)

In [6]:
pd.options.display.max_colwidth = 100
print("Vocabulary size:", len(wordvec_model.wv.vocab))
print("Number of reviews: ",len(traindf))
#for word in wordvec_model.wv.vocab:
#    print((word, wordvec_model.wv.vocab[word].count))
wordvec_model.wv.most_similar("great")

Vocabulary size: 94935
Number of reviews:  25000


[('terrific', 0.6434566974639893),
 ('tremendous', 0.6075615882873535),
 ('one_favourites', 0.6047403812408447),
 ('totally_awesome', 0.6020132303237915),
 ('wonderfully_do', 0.5982685089111328),
 ('greatthe', 0.5954743027687073),
 ('absolutely_incredible', 0.5937920808792114),
 ('fantastic', 0.5927061438560486),
 ('phenomenal', 0.5917737483978271),
 ('wonderful', 0.588508129119873)]

In [7]:
train_reviews = revcorpus[revcorpus['type']=="train"]["processed_review"]
train_reviews = trigrams[bigrams[train_reviews]]
train_reviews = [" ".join(text) for text in [review for review in train_reviews]]
print(len(train_reviews))

25000


In [8]:
 train_reviews[1]

'classic war_worlds timothy_hines entertain film obviously go great effort lengths faithfully recreate h_g_well classic book mr_hines succeed watch film appreciate fact standard predictable hollywood_fare come every_year eg spielberg version tom_cruise slightest resemblance book obviously everyone look different things movie envision amateur critics look criticize everything others rate movie important baseslike entertain people never agree critics enjoy effort mr_hines put faithful hg_well classic_novel find entertain make easy_overlook critics perceive shortcomings'

In [9]:
# Code grabbed from https://www.kaggle.com/alexcherniuk/imdb-review-word2vec-bilstm-99-acc
from keras.preprocessing.sequence import pad_sequences
def vectorize_data(data, vocab: dict) -> list:
    print('Vectorize sentences...', end='\r')
    keys = list(vocab.keys())
    filter_unknown = lambda word: vocab.get(word, None) is not None
    encode = lambda review: list(map(keys.index, filter(filter_unknown, review)))
    vectorized = list(map(encode, data))
    print('Vectorize sentences... (done)')
    return vectorized
embed_size = 150
X_pad = pad_sequences(
    sequences=vectorize_data(train_reviews, vocab=wordvec_model.wv.vocab),
    maxlen=embed_size,
    padding='post')

Using TensorFlow backend.


Vectorize sentences... (done)


In [14]:
X_pad[1]

array([ 4477, 13240, 15855,   966, 15497,   966,  4813,  6755,   793,
       15855,  4813, 15855,   302,  4477, 15855,   793, 15497,  9502,
        9502, 15855,   302, 15855,  4477, 10718,  4477,  4687,  4813,
       15855, 15855,  4477,  4813,  4477, 15855,  6755,   302, 13240,
         302, 13240,   793, 15497, 13240,   302, 13240, 11719, 15855,
       15855,  4813, 15855,   302, 10718,  4477,  4687, 10718, 15855,
         302,   302, 15855,  4813, 15855,  9396,   302,  4477,   966,
       15855,   793, 15497, 15855, 15855,  4477, 15855,   302,  4477,
        9396, 15855,  9396,   793, 15855,  4477, 15855,  4813, 15855,
         302,  4687,   302, 15855, 15855, 13240,   302, 13240, 15855,
        4477,  7056, 15855,  9502,  9502,   302,   302, 10718,  4477,
       15855,  9396,  6755,  9502, 10718,  9502,  6755,   793, 10718,
        4687,  9660, 15855,   793,   793, 13240,   793, 13240,  4477,
        4813, 15855,   793,  9502,  4477, 15855,  4477, 15855,   302,
        4477, 15497,

In [10]:
y = revcorpus[revcorpus['type']=="train"]['sentiment']

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten,CuDNNLSTM, CuDNNGRU
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

embed_size = 150
model = Sequential()
model.add(Embedding(input_dim=wordvec_model.wv.vectors.shape[0],output_dim=wordvec_model.wv.vectors.shape[1],weights=[wordvec_model.wv.vectors], input_length=embed_size))
#model.add(Embedding(max_features,embed_size))
model.add(Bidirectional(CuDNNGRU(32,return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(20,activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.summary()



W1005 14:45:56.307834 15752 deprecation_wrapper.py:119] From C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1005 14:45:56.326642 15752 deprecation_wrapper.py:119] From C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1005 14:45:56.330636 15752 deprecation_wrapper.py:119] From C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1005 14:45:56.346592 15752 deprecation_wrapper.py:119] From C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_sessio

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 300)          28480500  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 150, 64)           64128     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1300      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total para

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, shuffle=True)
X_test

array([[  302, 15855,  6755, ..., 13240,  6755,  4477],
       [15855,   302,   793, ...,  4477,   302, 15855],
       [15855,  4813, 15855, ...,  9660,   793, 15497],
       ...,
       [ 4477, 15855,  4477, ...,   793,  4813, 15855],
       [  793, 15497, 15855, ..., 13240, 13240,  4477],
       [ 4477,  4813,  6755, ...,   302,  6755,  4477]])

In [13]:
batch_size = 32
epochs = 3
model.compile(loss='binary_crossentropy', optimizer='AdaGrad', metrics=['accuracy'])
model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs)

W1005 14:46:01.668599 15752 deprecation_wrapper.py:119] From C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1005 14:46:01.698519 15752 deprecation.py:323] From C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5


UnknownError: 2 root error(s) found.
  (0) Unknown: Fail to find the dnn implementation.
	 [[{{node bidirectional_1/CudnnRNN_1}}]]
	 [[loss/mul/_95]]
  (1) Unknown: Fail to find the dnn implementation.
	 [[{{node bidirectional_1/CudnnRNN_1}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
# validate the model on test dataset to determine generalization
loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

In [ ]:
testdf=pd.read_csv("testData.tsv",header=0, delimiter="\t", quoting=3)
testdf.head(3)
